In [ ]:
# import torch
# import torch.nn as nn


# def Normalize(in_channels, num_groups=32):
#     return torch.nn.GroupNorm(num_groups=num_groups, num_channels=in_channels, eps=1e-6, affine=True)

# def nonlinearity(x):
#     # swish
#     return x*torch.sigmoid(x)

# # def make_attn(in_channels, attn_type="vanilla", attn_kwargs=None):
# #     assert attn_type in ["vanilla", "vanilla-xformers", "memory-efficient-cross-attn", "linear", "none"], f'attn_type {attn_type} unknown'
# #     if XFORMERS_IS_AVAILBLE and attn_type == "vanilla":
# #         attn_type = "vanilla-xformers"
# #     print(f"making attention of type '{attn_type}' with {in_channels} in_channels")
# #     if attn_type == "vanilla":
# #         assert attn_kwargs is None
# #         return AttnBlock(in_channels)
# #     elif attn_type == "vanilla-xformers":
# #         print(f"building MemoryEfficientAttnBlock with {in_channels} in_channels...")
# #         return MemoryEfficientAttnBlock(in_channels)
# #     elif type == "memory-efficient-cross-attn":
# #         attn_kwargs["query_dim"] = in_channels
# #         return MemoryEfficientCrossAttentionWrapper(**attn_kwargs)
# #     elif attn_type == "none":
# #         return nn.Identity(in_channels)
# #     else:
# #         raise NotImplementedError()
    
# class Upsample(nn.Module):
#     def __init__(self, in_channels, with_conv):
#         super().__init__()
#         self.with_conv = with_conv
#         if self.with_conv:
#             self.conv = torch.nn.Conv2d(in_channels,
#                                         in_channels,
#                                         kernel_size=3,
#                                         stride=1,
#                                         padding=1)

#     def forward(self, x):
#         x = torch.nn.functional.interpolate(x, scale_factor=2.0, mode="nearest")
#         if self.with_conv:
#             x = self.conv(x)
#         return x

    
# class Downsample(nn.Module):
#     def __init__(self, in_channels, with_conv):
#         super().__init__()
#         self.with_conv = with_conv
#         if self.with_conv:
#             # no asymmetric padding in torch conv, must do it ourselves
#             self.conv = torch.nn.Conv2d(in_channels,
#                                         in_channels,
#                                         kernel_size=3,
#                                         stride=2,
#                                         padding=0)

#     def forward(self, x):
#         if self.with_conv:
#             pad = (0,1,0,1)
#             x = torch.nn.functional.pad(x, pad, mode="constant", value=0)
#             x = self.conv(x)
#         else:
#             x = torch.nn.functional.avg_pool2d(x, kernel_size=2, stride=2)
#         return x

# class ResnetBlock(nn.Module):
#     def __init__(self, *, in_channels, out_channels=None, conv_shortcut=False,
#                  dropout, temb_channels=512):
#         super().__init__()
#         self.in_channels = in_channels
#         out_channels = in_channels if out_channels is None else out_channels
#         self.out_channels = out_channels
#         self.use_conv_shortcut = conv_shortcut

#         self.norm1 = Normalize(in_channels)
#         self.conv1 = torch.nn.Conv2d(in_channels,
#                                      out_channels,
#                                      kernel_size=3,
#                                      stride=1,
#                                      padding=1)
#         if temb_channels > 0:
#             self.temb_proj = torch.nn.Linear(temb_channels,
#                                              out_channels)
#         self.norm2 = Normalize(out_channels)
#         self.dropout = torch.nn.Dropout(dropout)
#         self.conv2 = torch.nn.Conv2d(out_channels,
#                                      out_channels,
#                                      kernel_size=3,
#                                      stride=1,
#                                      padding=1)
#         if self.in_channels != self.out_channels:
#             if self.use_conv_shortcut:
#                 self.conv_shortcut = torch.nn.Conv2d(in_channels,
#                                                      out_channels,
#                                                      kernel_size=3,
#                                                      stride=1,
#                                                      padding=1)
#             else:
#                 self.nin_shortcut = torch.nn.Conv2d(in_channels,
#                                                     out_channels,
#                                                     kernel_size=1,
#                                                     stride=1,
#                                                     padding=0)

#     def forward(self, x, temb):
#         h = x
#         h = self.norm1(h)
#         h = nonlinearity(h)
#         h = self.conv1(h)

#         if temb is not None:
#             h = h + self.temb_proj(nonlinearity(temb))[:,:,None,None]

#         h = self.norm2(h)
#         h = nonlinearity(h)
#         h = self.dropout(h)
#         h = self.conv2(h)

#         if self.in_channels != self.out_channels:
#             if self.use_conv_shortcut:
#                 x = self.conv_shortcut(x)
#             else:
#                 x = self.nin_shortcut(x)

#         return x+h


# class Encoder(nn.Module):
#     def __init__(self, *, ch, out_ch, ch_mult=(1,2,4,8), num_res_blocks,
#                  attn_resolutions, dropout=0.0, resamp_with_conv=True, in_channels,
#                  resolution, z_channels, double_z=True, use_linear_attn=False, attn_type="vanilla",
#                  **ignore_kwargs):
#         super().__init__()
#         if use_linear_attn: attn_type = "linear"
#         self.ch = ch
#         self.temb_ch = 0
#         self.num_resolutions = len(ch_mult)
#         self.num_res_blocks = num_res_blocks
#         self.resolution = resolution
#         self.in_channels = in_channels

#         # downsampling
#         self.conv_in = torch.nn.Conv2d(in_channels,
#                                        self.ch,
#                                        kernel_size=3,
#                                        stride=1,
#                                        padding=1)

#         curr_res = resolution
#         in_ch_mult = (1,)+tuple(ch_mult)
#         self.in_ch_mult = in_ch_mult
#         self.down = nn.ModuleList()
#         for i_level in range(self.num_resolutions):
#             block = nn.ModuleList()
#             attn = nn.ModuleList()
#             block_in = ch*in_ch_mult[i_level]
#             block_out = ch*ch_mult[i_level]
#             for i_block in range(self.num_res_blocks):
#                 block.append(ResnetBlock(in_channels=block_in,
#                                          out_channels=block_out,
#                                          temb_channels=self.temb_ch,
#                                          dropout=dropout))
#                 block_in = block_out
#                 if curr_res in attn_resolutions:
#                     attn.append(make_attn(block_in, attn_type=attn_type))
#             down = nn.Module()
#             down.block = block
#             down.attn = attn
#             if i_level != self.num_resolutions-1:
#                 down.downsample = Downsample(block_in, resamp_with_conv)
#                 curr_res = curr_res // 2
#             self.down.append(down)

#         # middle
#         self.mid = nn.Module()
#         self.mid.block_1 = ResnetBlock(in_channels=block_in,
#                                        out_channels=block_in,
#                                        temb_channels=self.temb_ch,
#                                        dropout=dropout)
#         self.mid.attn_1 = make_attn(block_in, attn_type=attn_type)
#         self.mid.block_2 = ResnetBlock(in_channels=block_in,
#                                        out_channels=block_in,
#                                        temb_channels=self.temb_ch,
#                                        dropout=dropout)

#         # end
#         self.norm_out = Normalize(block_in)
#         self.conv_out = torch.nn.Conv2d(block_in,
#                                         2*z_channels if double_z else z_channels,
#                                         kernel_size=3,
#                                         stride=1,
#                                         padding=1)

#     def forward(self, x):
#         # timestep embedding
#         temb = None

#         # downsampling
#         hs = [self.conv_in(x)]
#         for i_level in range(self.num_resolutions):
#             for i_block in range(self.num_res_blocks):
#                 h = self.down[i_level].block[i_block](hs[-1], temb)
#                 if len(self.down[i_level].attn) > 0:
#                     h = self.down[i_level].attn[i_block](h)
#                 hs.append(h)
#             if i_level != self.num_resolutions-1:
#                 hs.append(self.down[i_level].downsample(hs[-1]))

#         # middle
#         h = hs[-1]
#         h = self.mid.block_1(h, temb)
#         h = self.mid.attn_1(h)
#         h = self.mid.block_2(h, temb)

#         # end
#         h = self.norm_out(h)
#         h = nonlinearity(h)
#         h = self.conv_out(h)
#         return h


# class Decoder(nn.Module):
#     def __init__(self, *, ch, out_ch, ch_mult=(1,2,4,8), num_res_blocks,
#                  attn_resolutions, dropout=0.0, resamp_with_conv=True, in_channels,
#                  resolution, z_channels, give_pre_end=False, tanh_out=False, use_linear_attn=False,
#                  attn_type="vanilla", **ignorekwargs):
#         super().__init__()
#         if use_linear_attn: attn_type = "linear"
#         self.ch = ch
#         self.temb_ch = 0
#         self.num_resolutions = len(ch_mult)
#         self.num_res_blocks = num_res_blocks
#         self.resolution = resolution
#         self.in_channels = in_channels
#         self.give_pre_end = give_pre_end
#         self.tanh_out = tanh_out

#         # compute in_ch_mult, block_in and curr_res at lowest res
#         in_ch_mult = (1,)+tuple(ch_mult)
#         block_in = ch*ch_mult[self.num_resolutions-1]
#         curr_res = resolution // 2**(self.num_resolutions-1)
#         self.z_shape = (1,z_channels,curr_res,curr_res)
#         print("Working with z of shape {} = {} dimensions.".format(
#             self.z_shape, np.prod(self.z_shape)))

#         # z to block_in
#         self.conv_in = torch.nn.Conv2d(z_channels,
#                                        block_in,
#                                        kernel_size=3,
#                                        stride=1,
#                                        padding=1)

#         # middle
#         self.mid = nn.Module()
#         self.mid.block_1 = ResnetBlock(in_channels=block_in,
#                                        out_channels=block_in,
#                                        temb_channels=self.temb_ch,
#                                        dropout=dropout)
#         self.mid.attn_1 = make_attn(block_in, attn_type=attn_type)
#         self.mid.block_2 = ResnetBlock(in_channels=block_in,
#                                        out_channels=block_in,
#                                        temb_channels=self.temb_ch,
#                                        dropout=dropout)

#         # upsampling
#         self.up = nn.ModuleList()
#         for i_level in reversed(range(self.num_resolutions)):
#             block = nn.ModuleList()
#             attn = nn.ModuleList()
#             block_out = ch*ch_mult[i_level]
#             for i_block in range(self.num_res_blocks+1):
#                 block.append(ResnetBlock(in_channels=block_in,
#                                          out_channels=block_out,
#                                          temb_channels=self.temb_ch,
#                                          dropout=dropout))
#                 block_in = block_out
#                 if curr_res in attn_resolutions:
#                     attn.append(make_attn(block_in, attn_type=attn_type))
#             up = nn.Module()
#             up.block = block
#             up.attn = attn
#             if i_level != 0:
#                 up.upsample = Upsample(block_in, resamp_with_conv)
#                 curr_res = curr_res * 2
#             self.up.insert(0, up) # prepend to get consistent order

#         # end
#         self.norm_out = Normalize(block_in)
#         self.conv_out = torch.nn.Conv2d(block_in,
#                                         out_ch,
#                                         kernel_size=3,
#                                         stride=1,
#                                         padding=1)

#     def forward(self, z):
#         #assert z.shape[1:] == self.z_shape[1:]
#         self.last_z_shape = z.shape

#         # timestep embedding
#         temb = None

#         # z to block_in
#         h = self.conv_in(z)

#         # middle
#         h = self.mid.block_1(h, temb)
#         h = self.mid.attn_1(h)
#         h = self.mid.block_2(h, temb)

#         # upsampling
#         for i_level in reversed(range(self.num_resolutions)):
#             for i_block in range(self.num_res_blocks+1):
#                 h = self.up[i_level].block[i_block](h, temb)
#                 if len(self.up[i_level].attn) > 0:
#                     h = self.up[i_level].attn[i_block](h)
#             if i_level != 0:
#                 h = self.up[i_level].upsample(h)

#         # end
#         if self.give_pre_end:
#             return h

#         h = self.norm_out(h)
#         h = nonlinearity(h)
#         h = self.conv_out(h)
#         if self.tanh_out:
#             h = torch.tanh(h)
#         return h


In [109]:
import jax
import jax.numpy as jnp
from flax import linen as nn

def Normalize(num_groups=10):
    return nn.GroupNorm(num_groups=num_groups, epsilon=1e-6, use_scale=True)

class Downsample(nn.Module):
    in_channels: int
    
    def setup(self):
        self.conv = nn.Conv(
            self.in_channels,
            kernel_size=(3, 3),
            strides=(2, 2),
            padding=((0, 1), (0, 1)),
            )

    def __call__(self, x):
        pad = ((0, 0), (0, 1), (0, 0), (0, 1))
        x = jnp.pad(x, pad, mode="constant", constant_values=0)
        x = self.conv(x)
        return x

class ResnetBlock(nn.Module):
    in_channels: int
    out_channels: int
    
    def setup(self):

        self.norm1 = Normalize(num_groups=5)
        self.conv1 = nn.Conv(
            self.out_channels,
            kernel_size=(3, 3),
            strides=(1, 1),
            padding=((1, 1), (1, 1)),
        )
        self.norm2 = Normalize(num_groups=5)
        self.conv2 = nn.Conv(
            self.out_channels,
            kernel_size=(3, 3),
            strides=(1, 1),
            padding=((1, 1), (1, 1)),
        )

        self.nin_shortcut = nn.Conv(
            self.out_channels,
            kernel_size=(1, 1),
            strides=(1, 1),
            padding=((0, 0), (0, 0)),
        )

    def __call__(self, x):
        h = x
        h = self.norm1(h)
        h = nn.swish(h)
        h = self.conv1(h)
        h = self.norm2(h)
        h = nn.swish(h)
        h = self.conv2(h)
        x = self.nin_shortcut(x)
        x_ = x+h 
        
        return x + h
    
class DownsamplingBlock(nn.Module):
    ch: int
    ch_mult: tuple
    num_res_blocks: int
    resolution: int
    block_idx: int
    
    def setup(self):
        self.ch_mult_ = self.ch_mult 
        self.num_resolutions = len(self.ch_mult_)
        in_ch_mult = (1, ) + tuple(self.ch_mult_)
        block_in = self.ch * in_ch_mult[self.block_idx]
        block_out = self.ch * self.ch_mult_[self.block_idx]

        res_blocks = []
        for _ in range(self.num_res_blocks):
            res_blocks.append(ResnetBlock(block_in,
                                          block_out,
                                         ))
        block_in = block_out
        self.block = res_blocks

        self.downsample = None
        if self.block_idx != self.num_resolutions - 1:
            self.downsample = Downsample(block_in)

    def __call__(self, h):
        for i, res_block in enumerate(self.block):
            h = res_block(h)

        if self.downsample is not None:
            h = self.downsample(h)

        return h    

    
class MidBlock(nn.Module):
    in_channels : int
    
    def setup(self):
        self.block_1 = ResnetBlock(
            self.in_channels,
            self.in_channels,
        )
        self.block_2 = ResnetBlock(
            self.in_channels,
            self.in_channels,
        )
        
    def __call__(self, h):
        h = self.block_1(h)
        h = self.block_2(h)
        
        return h

class Encoder(nn.Module):
    ch: int
    out_ch: int
    ch_mult: tuple
    num_res_blocks: int
    in_channels: int
    resolution: int
    z_channels: int
    double_z: bool


    def setup(self):
        
        self.num_resolutions = len(self.ch_mult)
        
        # downsampling
        self.conv_in = nn.Conv(
            self.ch,
            kernel_size=(3, 3),
            strides=(1, 1),
            padding=((1, 1), (1, 1)),
        )
        
        curr_res = self.resolution
        downsample_blocks = []
        
        for i_level in range(self.num_resolutions):
            downsample_blocks.append(
                DownsamplingBlock(ch=self.ch,
                                  ch_mult=self.ch_mult,
                                  num_res_blocks=self.num_res_blocks,
                                  resolution=self.resolution,
                                  block_idx=i_level,
                                 ))
            if i_level != self.num_resolutions - 1:
                curr_res = curr_res // 2
                        
        self.down = downsample_blocks

        # middle
        mid_channels = self.ch * self.ch_mult[-1]
        self.mid = MidBlock(mid_channels)
        # end
        self.norm_out = Normalize()
        self.conv_out = nn.Conv(
            self.z_channels * 2 if self.double_z else self.z_channels,
            kernel_size=(3, 3),
            strides=(1, 1),
            padding=((1, 1), (1, 1)),
        )

    def __call__(self, x):

        # downsampling
        print("x :", x.shape)
        hs = self.conv_in(x)
        print("Conv_in :", hs.shape)
        for block in self.down:
            hs = block(hs)
        print("Down :", hs.shape)

        # middle
        hs = self.mid(hs)
        print("Mid :", hs.shape)
        
        # end
        hs = self.norm_out(hs)
        hs = nn.swish(hs)
        hs = self.conv_out(hs)
        print("Conv_out :", hs.shape)
        return hs

In [110]:
# Example usage
input_shape = (1, 64, 64, 5)  # (batch_size, height, width, channels)

# Create an instance of Downsample
downsample = Downsample(in_channels=5)

# Generate random input
rng = jax.random.PRNGKey(0)
x = jax.random.normal(rng, input_shape)

# Initialize the module and apply it to the input
params = downsample.init(rng, x)
output = downsample.apply(params, x)

# Print the output shape
print(output.shape)

print(Downsample(in_channels=5).tabulate(rng,x))

(1, 32, 32, 5)

                               Downsample Summary                               
┏━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ path ┃ module     ┃ inputs            ┃ outputs          ┃ params            ┃
┡━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│      │ Downsample │ float32[1,64,64,… │ float32[1,32,32… │                   │
├──────┼────────────┼───────────────────┼──────────────────┼───────────────────┤
│ conv │ Conv       │ float32[1,65,64,… │ float32[1,32,32… │ bias: float32[5]  │
│      │            │                   │                  │ kernel:           │
│      │            │                   │                  │ float32[3,3,6,5]  │
│      │            │                   │                  │                   │
│      │            │                   │                  │ 275 (1.1 KB)      │
├──────┼────────────┼───────────────────┼──────────────────┼───────────────────┤
│      │    

In [111]:
from clu import parameter_overview

# Generate random input tensors
x = jnp.zeros((1, 64, 64, 5))
rng = jax.random.PRNGKey(0)

# Initialize DownsamplingBlock
down_block = DownsamplingBlock(
    ch=5,
    ch_mult=(1,2,4),
    num_res_blocks=2,
    resolution=64,
    block_idx=1,
)

params = down_block.init(rng, x)
# Apply DownsamplingBlock using the apply method
output = down_block.apply({'params': params['params']}, x)
print(output.shape)

print(down_block.tabulate(rng, x))

print(parameter_overview.get_parameter_overview(params))

(1, 32, 32, 10)

                           DownsamplingBlock Summary                            
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ path          ┃ module        ┃ inputs        ┃ outputs       ┃ params       ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│               │ Downsampling… │ float32[1,64… │ float32[1,32… │              │
├───────────────┼───────────────┼───────────────┼───────────────┼──────────────┤
│ block_0       │ ResnetBlock   │ float32[1,64… │ float32[1,64… │              │
├───────────────┼───────────────┼───────────────┼───────────────┼──────────────┤
│ block_0/norm1 │ GroupNorm     │ float32[1,64… │ float32[1,64… │ bias:        │
│               │               │               │               │ float32[5]   │
│               │               │               │               │ scale:       │
│               │               │               │               │ float32[5]   │
│          

In [112]:
# Generate random input tensors
x = jnp.zeros((1, 64, 64, 5))
rng = jax.random.PRNGKey(0)

# Initialize ResnetBlock
resnet_block = ResnetBlock(
    in_channels=5,
    out_channels=5,
)

params = resnet_block.init(rng, x)
# Apply ResnetBlock using the apply method
output = resnet_block.apply({'params': params['params']}, x)
print(output.shape)

print(resnet_block.tabulate(rng, x))

print(parameter_overview.get_parameter_overview(params))

(1, 64, 64, 5)

                              ResnetBlock Summary                               
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ path         ┃ module      ┃ inputs         ┃ outputs        ┃ params        ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│              │ ResnetBlock │ float32[1,64,… │ float32[1,64,… │               │
├──────────────┼─────────────┼────────────────┼────────────────┼───────────────┤
│ norm1        │ GroupNorm   │ float32[1,64,… │ float32[1,64,… │ bias:         │
│              │             │                │                │ float32[5]    │
│              │             │                │                │ scale:        │
│              │             │                │                │ float32[5]    │
│              │             │                │                │               │
│              │             │                │                │ 10 (40 B)     │
├───────────

In [113]:
# Example usage
rng = jax.random.PRNGKey(0)
input_shape = (1, 64, 64, 5) 

# Initialize Encoder
encoder = Encoder(ch_mult=(1,2,4),
                  num_res_blocks=2,
                  double_z=False,
                  z_channels=5,
                  resolution=64,
                  in_channels=5,
                  out_ch=5,
                  ch=5,                  
                 )

# Apply Encoder
params = encoder.init(rng, x=jnp.ones(input_shape))
output = encoder.apply(params, x=jnp.ones(input_shape))

# Print the output shape
print("Encoder output :", output.shape)  # Shape: (1, 16, 16, 5)

x : (1, 64, 64, 5)
Conv_in : (1, 64, 64, 5)
Down : (1, 16, 16, 20)
Mid : (1, 16, 16, 20)
Conv_out : (1, 16, 16, 5)
x : (1, 64, 64, 5)
Conv_in : (1, 64, 64, 5)
Down : (1, 16, 16, 20)
Mid : (1, 16, 16, 20)
Conv_out : (1, 16, 16, 5)
Encoder output : (1, 16, 16, 5)


In [114]:
class Upsample(nn.Module):
    in_channels: int
    
    def setup(self):
        self.conv = nn.Conv(
            self.in_channels,
            kernel_size=(3, 3),
            strides=(1, 1),
            padding=((1, 1), (1, 1)),
            )

    def __call__(self, hs):
        batch, height, width, channels = hs.shape
        hs = jax.image.resize(
            hs,
            shape=(batch, height * 2, width * 2, channels),
            method="nearest",
        )
        hs = self.conv(hs)
        return hs
    
    
class UpsamplingBlock(nn.Module):
    ch: int
    ch_mult: tuple
    num_res_blocks: int
    resolution: int
    block_idx: int
    
    def setup(self):
        
        self.ch_mult_ = self.ch_mult 
        self.num_resolutions = len(self.ch_mult_)
        
        if self.block_idx == self.num_resolutions - 1:
            block_in = self.ch * self.ch_mult_[-1]
        else:
            block_in = self.ch * self.ch_mult_[self.block_idx + 1]

        block_out = self.ch * self.ch_mult_[self.block_idx]
        
        res_blocks = []
        for _ in range(self.num_res_blocks + 1):
            res_blocks.append(
                ResnetBlock(block_in,
                            block_out))
                            
        block_in = block_out
        
        self.block = res_blocks

        self.upsample = None
        if self.block_idx != 0:
            self.upsample = Upsample(block_in)

    def __call__(self, h):
        for i, res_block in enumerate(self.block):
            h = res_block(h)

        if self.upsample is not None:
            h = self.upsample(h)

        return h    


In [115]:
from clu import parameter_overview

# Generate random input tensors
x = jnp.zeros((1, 16, 16, 5))
rng = jax.random.PRNGKey(0)

# Initialize UpsamplingBlock
up_block = UpsamplingBlock(
    ch=5,
    ch_mult=(4,2,1),
    num_res_blocks=2,
    resolution=64,
    block_idx=2,
)

params = up_block.init(rng, x)
# Apply UpsamplingBlock using the apply method
output = up_block.apply({'params': params['params']}, x)
print(output.shape)

print(up_block.tabulate(rng, x))

print(parameter_overview.get_parameter_overview(params))

(1, 32, 32, 5)

                            UpsamplingBlock Summary                             
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ path          ┃ module        ┃ inputs        ┃ outputs       ┃ params       ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│               │ UpsamplingBl… │ float32[1,16… │ float32[1,32… │              │
├───────────────┼───────────────┼───────────────┼───────────────┼──────────────┤
│ block_0       │ ResnetBlock   │ float32[1,16… │ float32[1,16… │              │
├───────────────┼───────────────┼───────────────┼───────────────┼──────────────┤
│ block_0/norm1 │ GroupNorm     │ float32[1,16… │ float32[1,16… │ bias:        │
│               │               │               │               │ float32[5]   │
│               │               │               │               │ scale:       │
│               │               │               │               │ float32[5]   │
│           

In [119]:
import numpy as np

class Decoder(nn.Module):
    ch: int
    out_ch: int
    ch_mult: tuple
    num_res_blocks: int
    in_channels: int
    resolution: int
    z_channels: int
    double_z: bool
    
    def setup(self):
        
        self.num_resolutions = len(self.ch_mult)
        
        block_in = self.ch*self.ch_mult[self.num_resolutions-1]
        curr_res = self.resolution // 2**(self.num_resolutions-1)
        self.z_shape = (1,self.z_channels,curr_res,curr_res)
        
        # z to block_in
        self.conv_in = nn.Conv(
            self.ch,
            kernel_size=(3, 3),
            strides=(1, 1),
            padding=((1, 1), (1, 1)),
        )
               
        print("Working with z of shape {} = {} dimensions.".format(
        self.z_shape, np.prod(self.z_shape)))
        
        # middle
        self.mid = MidBlock(block_in)
        
        # upsampling
        upsample_blocks = []
        
        for i_level in reversed(range(self.num_resolutions)):
            upsample_blocks.append(
                UpsamplingBlock(ch=self.ch,
                                ch_mult=self.ch_mult,
                                num_res_blocks=self.num_res_blocks,
                                resolution=self.resolution,
                                block_idx=i_level,
                               ))
            if i_level != 0:
                curr_res = curr_res * 2
        self.up = list(
            reversed(upsample_blocks))  # reverse to get consistent order
        
        # end
        self.norm_out = Normalize(num_groups=5)
        self.conv_out = nn.Conv(
            self.out_ch,
            kernel_size=(3, 3),
            strides=(1, 1),
            padding=((1, 1), (1, 1)),
        )
        
    def __call__(self, z):
        # z to block_in
        hs = self.conv_in(z)
        
        # middle
        hs = self.mid(hs)
        
        # upsampling
        for block in reversed(self.up):
            hs = block(hs)
            
        # end
        hs = self.norm_out(hs)
        hs = nn.swish(hs)
        hs = self.conv_out(hs)
        
        return hs

In [120]:
# Example usage
rng = jax.random.PRNGKey(0)
input_shape = (1, 16, 16, 5) 

# Initialize ResnetBlock
decoder = Decoder(ch_mult=(1,2,4),
                  num_res_blocks=2,
                  double_z=False,
                  z_channels=5,
                  resolution=64,
                  in_channels=5,
                  out_ch=5,
                  ch=5,                  
                 )

# Apply Resnet
params = decoder.init(rng, z=jnp.ones(input_shape))
output = decoder.apply(params, z=jnp.ones(input_shape))

# Print the output shape
print("Decoder output :", output.shape)  # Shape: (1, 64, 64, 5)

Working with z of shape (1, 5, 16, 16) = 1280 dimensions.
Working with z of shape (1, 5, 16, 16) = 1280 dimensions.
Decoder output : (1, 64, 64, 5)
